In [8]:
#Read in data

import pandas as pd
import collections as co
import math
import numpy as np
import scipy.sparse as spr
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

bios = pd.read_csv("bios.csv", usecols = ["Bio","Label","Year","Name"])

In [9]:
#Clean up
n = bios.shape[0]

bios.Bio = bios.Bio.str.lower()

bios.Name = bios.Name.str.lower()

stemmer = SnowballStemmer("english")

for i in range(n):
    bios.Bio.loc[i] = nltk.word_tokenize(bios.Bio.loc[i])
    bios.Name.loc[i] = nltk.word_tokenize(bios.Name.loc[i])

for i in range(n):
    for j in range(len(bios.Bio.loc[i])):
        bios.Bio.loc[i][j] = stemmer.stem(bios.Bio.loc[i][j])
    
    if ("dr" in bios.Name.loc[i]):
        bios.Name.loc[i].remove("dr")
    if ("prof" in bios.Name.loc[i]):
        bios.Name.loc[i].remove("prof")
    
    if (bios.Label.loc[i] != 'F') or (bios.Label.loc[i] != 'M'):
        if ('F' in bios.Label.loc[i]):
            bios.loc[i,"Label"] = 'f'
        elif ('M' in bios.Label.loc[i]):
            bios.loc[i,"Label"] = 'm'

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [10]:
#Calculate document frequency for male and female bios
dict_base = co.Counter()
dict_all = co.Counter()
dict_m = co.Counter()
dict_f = co.Counter()
stop_words = set(stopwords.words('english'))

for j in range(len(bios.Bio.loc[i])):
    if (bios.Bio.loc[i][j] not in stop_words) and (bios.Bio.loc[i][j] not in bios.Name.loc[i]) and (bios.Bio.loc[i][j].isalpha()):
        dict_base[bios.Bio.loc[i][j]] += 1

for i in range(n):
    docflag = dict_base.copy()
    if bios.Label.loc[i] == 'm':
        for j in range(len(bios.Bio.loc[i])):
            if (bios.Bio.loc[i][j] not in stop_words) and (bios.Bio.loc[i][j] not in bios.Name.loc[i]) and (bios.Bio.loc[i][j].isalpha()):
                if (docflag[bios.Bio.loc[i][j]] > -1):
                    dict_m[bios.Bio.loc[i][j]] += 1
                    dict_all[bios.Bio.loc[i][j]] += 1
                    docflag[bios.Bio.loc[i][j]] = -1
    else:
        for j in range(len(bios.Bio.loc[i])):
            if (bios.Bio.loc[i][j] not in stop_words) and (bios.Bio.loc[i][j] not in bios.Name.loc[i]) and (bios.Bio.loc[i][j].isalpha()):
                if (docflag[bios.Bio.loc[i][j]] > -1):
                    dict_f[bios.Bio.loc[i][j]] += 1
                    dict_all[bios.Bio.loc[i][j]] += 1
                    docflag[bios.Bio.loc[i][j]] = -1

print("Top 10 word stems:")
print(pd.DataFrame(dict_all.most_common()[:10]))
print()
print("Top 10 word stems for males:")
print(pd.DataFrame(dict_m.most_common()[:10]))
print()
print("Top 10 word stems for females:")
print(pd.DataFrame(dict_f.most_common()[:10]))

Top 10 word stems:
           0    1
0   research  154
1    univers  145
2     scienc  137
3     comput  136
4  professor  121
5     receiv   77
6     includ   76
7       work   69
8     system   68
9      award   68

Top 10 word stems for males:
           0    1
0   research  121
1    univers  115
2     scienc  112
3     comput  109
4  professor   93
5     includ   65
6       work   60
7     receiv   58
8   institut   55
9      award   54

Top 10 word stems for females:
           0   1
0   research  33
1    univers  30
2  professor  28
3     comput  27
4     scienc  25
5     receiv  19
6     associ  16
7      engin  16
8     fellow  15
9   interest  14
